In [1]:
import pandas as pd
import numpy as np

#Take file name as input
user_in = input("Enter the source filename (without the csv extension): ")
filename = user_in +'.csv'

#Read file
df = pd.read_csv(filename,
                 names = ['Player', 'KPI Set', 'Exercise Type', 'Exercise Name', 'Review Type',
                          'Comment Q1', 'Comment Q2', 'FRAMING', 'VALIDATION', 'RELATIONAL',
                         'ARGUMENT', 'NON-VERBAL', 'PROCEDURAL', 'Overall'], header=0,engine='python')

#Split data frame into self and coach, calculate overall averages - going to merge eventually
df_self = df[df['Review Type'] == 'Self']
df_coach = df[df['Review Type'] == 'Coach']
df_self_means = df_self.groupby('Player').mean().reset_index()
df_coach_means = df_coach.groupby('Player').mean().reset_index()

#Put overall averages into data frames and remove 'overall' column
df_self_final = df_self_means
df_self_final = df_self_final.drop(columns=['Overall'])
df_self_final = df_self_final.rename(columns = {'FRAMING':'F_Self', 'VALIDATION':'V_Self', 'RELATIONAL':'R_Self',
                                               'ARGUMENT':'A_Self', 'NON-VERBAL':'N_Self', 'PROCEDURAL':'P_Self'})
df_coach_final = df_coach_means
df_coach_final = df_coach_final.drop(columns=['Overall'])
df_coach_final = df_coach_final.rename(columns = {'FRAMING':'F_E_Self', 'VALIDATION':'V_E_Self', 'RELATIONAL':'R_E_Self',
                                               'ARGUMENT':'A_E_Self', 'NON-VERBAL':'N_E_Self', 'PROCEDURAL':'P_E_Self'})


#Calculate peer averages
#Iterate through rows, drop current row, calc average of other rows
F_peer = []
V_peer = []
R_peer = []
A_peer = []
N_peer = []
P_peer = []
for i in range(df_self_final.shape[0]):
    inter_df = df_self_final.drop([i], axis = 0)
    inter_means = inter_df.mean()
    F_peer.append(inter_means['F_Self'])
    V_peer.append(inter_means['V_Self'])
    R_peer.append(inter_means['R_Self'])
    A_peer.append(inter_means['A_Self'])
    N_peer.append(inter_means['N_Self'])
    P_peer.append(inter_means['P_Self'])

F_E_peer = []
V_E_peer = []
R_E_peer = []
A_E_peer = []
N_E_peer = []
P_E_peer = []
for i in range(df_coach_final.shape[0]):
    inter_df = df_coach_final.drop([i], axis = 0)
    inter_means = inter_df.mean()
    F_E_peer.append(inter_means['F_E_Self'])
    V_E_peer.append(inter_means['V_E_Self'])
    R_E_peer.append(inter_means['R_E_Self'])
    A_E_peer.append(inter_means['A_E_Self'])
    N_E_peer.append(inter_means['N_E_Self'])
    P_E_peer.append(inter_means['P_E_Self'])

#Add peer averages to data frames
df_self_final['F_Peer'] = pd.Series(F_peer)
df_self_final['V_Peer'] = pd.Series(V_peer)
df_self_final['R_Peer'] = pd.Series(R_peer)
df_self_final['A_Peer'] = pd.Series(A_peer)
df_self_final['N_Peer'] = pd.Series(N_peer)
df_self_final['P_Peer'] = pd.Series(P_peer)

df_coach_final['F_E_Peer'] = pd.Series(F_E_peer)
df_coach_final['V_E_Peer'] = pd.Series(V_E_peer)
df_coach_final['R_E_Peer'] = pd.Series(R_E_peer)
df_coach_final['A_E_Peer'] = pd.Series(A_E_peer)
df_coach_final['N_E_Peer'] = pd.Series(N_E_peer)
df_coach_final['P_E_Peer'] = pd.Series(P_E_peer)


#Individual exercise breakdown 
#Self
exercises = df['Exercise Name'].unique()
for i, exercise in enumerate(exercises, 1):
    exer_df = df_self[df_self['Exercise Name'] == exercise]
    df_indiv_self = exer_df.groupby('Player').mean().reset_index()
    df_self_final[f'F_{i}_Self'] = df_indiv_self['FRAMING']
    df_self_final[f'V_{i}_Self'] = df_indiv_self['VALIDATION']
    df_self_final[f'R_{i}_Self'] = df_indiv_self['RELATIONAL']
    df_self_final[f'A_{i}_Self'] = df_indiv_self['ARGUMENT']
    df_self_final[f'N_{i}_Self'] = df_indiv_self['NON-VERBAL']
    df_self_final[f'P_{i}_Self'] = df_indiv_self['PROCEDURAL']
    
#Coach
for i, exercise in enumerate(exercises, 1):
    exer_df = df_coach[df_coach['Exercise Name'] == exercise]
    df_indiv_coach = exer_df.groupby('Player').mean().reset_index()
    df_coach_final[f'F_{i}_E_Self'] = df_indiv_coach['FRAMING']
    df_coach_final[f'V_{i}_E_Self'] = df_indiv_coach['VALIDATION']
    df_coach_final[f'R_{i}_E_Self'] = df_indiv_coach['RELATIONAL']
    df_coach_final[f'A_{i}_E_Self'] = df_indiv_coach['ARGUMENT']
    df_coach_final[f'N_{i}_E_Self'] = df_indiv_coach['NON-VERBAL']
    df_coach_final[f'P_{i}_E_Self'] = df_indiv_coach['PROCEDURAL']

#Individual exercise peer averages
#Self
F_peer_av = []
V_peer_av = []
R_peer_av = []
A_peer_av = []
N_peer_av = []
P_peer_av = []

F_peer_sd = []
V_peer_sd = []
R_peer_sd = []
A_peer_sd = []
N_peer_sd = []
P_peer_sd = []

lists = [F_peer_av, V_peer_av, R_peer_av, A_peer_av, N_peer_av, P_peer_av, F_peer_sd, V_peer_sd, R_peer_sd,
A_peer_sd, N_peer_sd, P_peer_sd]


for i, exercise in enumerate(exercises, 1):
    for l in lists:
        l.clear()
        
    exer_df = df_self_final[['Player', f'F_{i}_Self', f'V_{i}_Self', f'R_{i}_Self', f'A_{i}_Self',
                            f'N_{i}_Self', f'P_{i}_Self']]
    
    for j in range(0, exer_df.shape[0]):
        inter_df = exer_df.drop([j], axis = 0)
        inter_means = inter_df.mean()
        inter_sd = inter_df.std()
        
        F_peer_av.append(inter_means[f'F_{i}_Self'])
        V_peer_av.append(inter_means[f'V_{i}_Self'])
        R_peer_av.append(inter_means[f'R_{i}_Self'])
        A_peer_av.append(inter_means[f'A_{i}_Self'])
        N_peer_av.append(inter_means[f'N_{i}_Self'])
        P_peer_av.append(inter_means[f'P_{i}_Self'])
        
        F_peer_sd.append(inter_sd[f'F_{i}_Self'])
        V_peer_sd.append(inter_sd[f'V_{i}_Self'])
        R_peer_sd.append(inter_sd[f'R_{i}_Self'])
        A_peer_sd.append(inter_sd[f'A_{i}_Self'])
        N_peer_sd.append(inter_sd[f'N_{i}_Self'])
        P_peer_sd.append(inter_sd[f'P_{i}_Self'])
        
    #Add averages and sd's to data frame
    df_self_final[f'F_{i}_Peer_Av'] = pd.Series(F_peer_av)
    df_self_final[f'V_{i}_Peer_Av'] = pd.Series(V_peer_av)
    df_self_final[f'R_{i}_Peer_Av'] = pd.Series(R_peer_av)
    df_self_final[f'A_{i}_Peer_Av'] = pd.Series(A_peer_av)
    df_self_final[f'N_{i}_Peer_Av'] = pd.Series(N_peer_av)
    df_self_final[f'P_{i}_Peer_Av'] = pd.Series(P_peer_av)
    
    df_self_final[f'F_{i}_Peer_SD'] = pd.Series(F_peer_sd)
    df_self_final[f'V_{i}_Peer_SD'] = pd.Series(V_peer_sd)
    df_self_final[f'R_{i}_Peer_SD'] = pd.Series(R_peer_sd)
    df_self_final[f'A_{i}_Peer_SD'] = pd.Series(A_peer_sd)
    df_self_final[f'N_{i}_Peer_SD'] = pd.Series(N_peer_sd)
    df_self_final[f'P_{i}_Peer_SD'] = pd.Series(P_peer_sd)


#Individual exercise peer averages
#Coach

for i, exercise in enumerate(exercises, 1):
    for l in lists:
        l.clear()
        
    exer_df = df_coach_final[['Player', f'F_{i}_E_Self', f'V_{i}_E_Self', f'R_{i}_E_Self', f'A_{i}_E_Self',
                            f'N_{i}_E_Self', f'P_{i}_E_Self']]
    
    for j in range(0, exer_df.shape[0]):
        inter_df = exer_df.drop([j], axis = 0)
        inter_means = inter_df.mean()
        inter_sd = inter_df.std()
        
        F_peer_av.append(inter_means[f'F_{i}_E_Self'])
        V_peer_av.append(inter_means[f'V_{i}_E_Self'])
        R_peer_av.append(inter_means[f'R_{i}_E_Self'])
        A_peer_av.append(inter_means[f'A_{i}_E_Self'])
        N_peer_av.append(inter_means[f'N_{i}_E_Self'])
        P_peer_av.append(inter_means[f'P_{i}_E_Self'])
        
        F_peer_sd.append(inter_sd[f'F_{i}_E_Self'])
        V_peer_sd.append(inter_sd[f'V_{i}_E_Self'])
        R_peer_sd.append(inter_sd[f'R_{i}_E_Self'])
        A_peer_sd.append(inter_sd[f'A_{i}_E_Self'])
        N_peer_sd.append(inter_sd[f'N_{i}_E_Self'])
        P_peer_sd.append(inter_sd[f'P_{i}_E_Self'])
        
    #Add averages and sd's to data frame
    df_coach_final[f'F_{i}_E_Peer_Av'] = pd.Series(F_peer_av)
    df_coach_final[f'V_{i}_E_Peer_Av'] = pd.Series(V_peer_av)
    df_coach_final[f'R_{i}_E_Peer_Av'] = pd.Series(R_peer_av)
    df_coach_final[f'A_{i}_E_Peer_Av'] = pd.Series(A_peer_av)
    df_coach_final[f'N_{i}_E_Peer_Av'] = pd.Series(N_peer_av)
    df_coach_final[f'P_{i}_E_Peer_Av'] = pd.Series(P_peer_av)
    
    df_coach_final[f'F_{i}_E_Peer_SD'] = pd.Series(F_peer_sd)
    df_coach_final[f'V_{i}_E_Peer_SD'] = pd.Series(V_peer_sd)
    df_coach_final[f'R_{i}_E_Peer_SD'] = pd.Series(R_peer_sd)
    df_coach_final[f'A_{i}_E_Peer_SD'] = pd.Series(A_peer_sd)
    df_coach_final[f'N_{i}_E_Peer_SD'] = pd.Series(N_peer_sd)
    df_coach_final[f'P_{i}_E_Peer_SD'] = pd.Series(P_peer_sd)


#Add individual comment columns
#Self

for i, exercise in enumerate(exercises, 1):
    exer_df = df_self[df_self['Exercise Name'] == exercise]
    df_indiv_self = exer_df.sort_values(by=['Player']).reset_index()
    df_self_final[f'Self_{i}_Q1'] = df_indiv_self['Comment Q1']
    df_self_final[f'Self_{i}_Q2'] = df_indiv_self['Comment Q2']
    
#Coach
for i, exercise in enumerate(exercises, 1):
    exer_df = df_coach[df_coach['Exercise Name'] == exercise]
    df_indiv_coach = exer_df.sort_values(by=['Player']).reset_index()
    df_coach_final[f'E_{i}_Q1'] = df_indiv_coach['Comment Q1']
    df_coach_final[f'E_{i}_Q2'] = df_indiv_coach['Comment Q2']

#Add Expert-Rated column
#Self
expert_col = df_self_final['Player'].isin(df_coach_final['Player']).astype(int)
df_self_final.insert(0, 'Expert_Rated', expert_col)

#Coach
df_coach_final['Expert_Rated'] = pd.Series(np.ones(df_coach_final.shape[0]).astype(int))

#Merge data frames
df_final = pd.merge(df_self_final, df_coach_final, on=['Player', 'Expert_Rated'], how='outer')

#Round values
df_final = df_final.round(decimals=2)


Enter the source filename (without the csv extension):  influence_data_fake


In [3]:
#Insert empty/dummy columns
new_cols = ["Language", "Unsubscribed", "ExternalDataReference", "Email", "LastName", "FirstName", "RecipientID"]
for col in new_cols:
    df_final.insert(loc=0, column = col, value = "")
df_final["Email"] = "sample@sample.edu"

In [ ]:
#Export
df_final.to_csv(user_in + '_finished.csv')

In [7]:
df_final

,RecipientID,FirstName,LastName,Email,ExternalDataReference,Unsubscribed,Language,Expert_Rated,Player,F_Self,...,E_2_Q1,E_2_Q2,E_3_Q1,E_3_Q2,E_4_Q1,E_4_Q2,E_5_Q1,E_5_Q2,E_6_Q1,E_6_Q2
0,,,,sample@sample.edu,,,,1,Abigail Dalton,3.17,...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...
1,,,,sample@sample.edu,,,,1,Adam Blauzvern,4.33,...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...
2,,,,sample@sample.edu,,,,0,Adrian Pereira,1.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,,,sample@sample.edu,,,,0,Alexander Medearis,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,,,sample@sample.edu,,,,1,Ally Howington,3.17,...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,,,,sample@sample.edu,,,,0,Troy Mitchell,4.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,,,,sample@sample.edu,,,,0,Won Kim,4.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,,,,sample@sample.edu,,,,1,Yakub Huda,2.67,...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...,this is the POSITIVE coach comment for IN07 - ...,this is the IMPROVEMENT coach comment for IN07...
62,,,,sample@sample.edu,,,,0,Yanxuan Xie,4.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
